In [14]:
%matplotlib inline

In [15]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import obspy
import os
from obspy.clients.fdsn import Client
from obspy.clients.syngine import Client as SyngineClient
from obspy.taup import TauPyModel
from obspy.geodetics import kilometer2degrees
from obspy.io.xseed import Parser
from obspy.signal.cross_correlation import xcorr,xcorr_3c

# Coletando os dados das estações ROSB e SMTB

In [16]:
ev_list = []
ev_listS = []

for root, dirs, files in os.walk('/home/diogo/dados_doutorado/parnaiba_basin/assessing_data/STA_data/'):
    for datafile in files:
        if datafile.startswith('BR.'):
            ev_list.append(os.path.join(root, datafile))
ev_listS = sorted(ev_list)

In [17]:
ev = obspy.Stream()
for i,j in enumerate(ev_listS):
    ev += obspy.read(j)

In [18]:
event_DD = []
event_MM = []
event_YYYY = []
event_hh = []
event_mm = []
event_julday = []
event_depth = []
event_lat = []
event_long = []
event_dist = []
event_sta = []
event_channel = []
sta_lat = []
sta_long = []
sta_channel = []
event_starttime = []
event_endtime = []
event_gcarc = []

for i,j in enumerate(ev):
    event_time = (j.stats.starttime)
    sta_channel.append(j.stats.channel[-1])
    event_starttime.append(j.stats.starttime)
    event_endtime.append(j.stats.endtime)
    event_DD.append("{0:02.0f}".format(event_time.day))
    event_MM.append("{0:02.0f}".format(event_time.month))
    event_YYYY.append(event_time.year)
    event_hh.append("{0:02.0f}".format(event_time.hour))
    event_mm.append("{0:02.0f}".format(event_time.minute))
    event_julday.append(event_time.julday)
    event_depth.append(j.stats.sac.evdp)
    event_lat.append(j.stats.sac.evla)
    event_long.append(j.stats.sac.evlo)
    event_dist.append(j.stats.sac.dist)
    event_sta.append(j.stats.sac.kstnm)
    event_gcarc.append(j.stats.sac.gcarc)
    event_channel.append(j.stats.sac.kcmpnm[-1])
    sta_lat.append(j.stats.sac.stla)
    sta_long.append(j.stats.sac.stlo)

In [19]:
eventid_lst = []
for k,l in enumerate(ev):
    eventid_lst.append('GCMT:C'+str(event_YYYY[k])+str(event_MM[k])+str(event_DD[k])+str(event_hh[k])+str(event_mm[k])+'A')

eventid_lst = sorted(list(set(eventid_lst)))

In [20]:
initial_time = [str(i) for i in event_starttime]
initial_time = sorted(list(set(initial_time)))
starttime_lst = [obspy.UTCDateTime(i) for i in initial_time] 

In [21]:
lat_lst = list(set(sta_lat))
long_lst = list(set(sta_long))
sta_lst = list(set(event_sta))

In [22]:
bulk_lst = []
for k,l in enumerate(sta_lst):
    bulk_lst.append({"latitude": lat_lst[k], "longitude": long_lst[k], "stationcode": sta_lst[k],"networkcode": 'BR'})

## Calculando os sismogramas sintéticos através do servidor da Syngine IRIS

In [23]:
%%time

c_s = SyngineClient()
#model = ["ak135f_2s", "iasp91_2s", "prem_a_2s"]
model = "ak135f_2s"


st_synth = []
for k,l in enumerate(eventid_lst):
    try:
        print('Evento '+l)
        st_synth.append(obspy.Stream(c_s.get_waveforms_bulk(model=model, 
                                                bulk=bulk_lst, 
                                                eventid=eventid_lst[k],
                                                dt="0.01",units="displacement",starttime="P-500", endtime="P+3000",
                                                format='saczip')))
    except:
        print('Problema no Evento: '+eventid_lst[k])

Evento GCMT:C201604162358A
Evento GCMT:C201605280946A
Problema no Evento: GCMT:C201605280946A
Evento GCMT:C201608190732A
Evento GCMT:C201608290429A
Problema no Evento: GCMT:C201608290429A
Evento GCMT:C201612251422A
CPU times: user 988 ms, sys: 196 ms, total: 1.18 s
Wall time: 1min 16s


In [24]:
syn = []
syn = [item for sublist in st_synth for item in sublist]

In [25]:
loc_folder = '/home/diogo/dados_doutorado/parnaiba_basin/assessing_data/SYN_data/'

In [26]:
%%time

for k,t in enumerate(syn):
        j = t.stats.starttime
        folder_loc_string = loc_folder+t.stats.station+'/'+str(j.year)+'/'+str("{0:0=3d}".format(j.julday))+'/'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)
        os.makedirs(folder_loc_string,exist_ok=True)
        t.write(folder_loc_string+'/SYN.'+t.stats.network+'.'+t.stats.station+'.'+str(j.year)+'.'+str(j.hour)+'.'+str(j.minute)+'.'+str(j.second)+'.'+t.stats.channel[-1],format='SAC')

CPU times: user 596 ms, sys: 8 ms, total: 604 ms
Wall time: 616 ms
